In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
#file path
path = '../data/bisi_enterprise_store_2022_2024.csv'

#pandas dataframe
df = pd.read_csv(path)

df.head()

,InvoiceNo,TransactionDate,City,Channel,CustomerType,Product,Category,Quantity,NominalUnitPrice,DiscountRate,NominalTotalValue,CPI_Index,RealUnitPrice,RealTotalValue,GrossMargin,PaymentType,PromoFlag,CustomerID,Year,Month
0,INV1000000,2023-02-15,Lagos,Physical,SME,Detergent,Household,1,1338.26,0.0,1338.26,129.714286,1031.70,1031.70,200.74,Transfer,False,CUST1813,2023,2023-02-01
1,INV1000001,2024-11-07,Port Harcourt,Physical,Student,Tecno Phone,Electronics,1,168556.49,0.0,168556.49,177.714286,94846.90,94846.90,30340.17,Cash,False,CUST7636,2024,2024-11-01
2,INV1000002,2024-03-12,Enugu,Physical,Reseller,Toilet Roll,Household,2,5961.54,0.2,9538.46,159.428571,3739.32,5982.91,1430.77,Cash,True,CUST5686,2024,2024-03-01
3,INV1000003,2023-10-18,Onitsha,Online,Student,Body Cream,Beauty,2,17362.08,0.1,31251.74,148.000000,11731.14,21116.05,6875.38,POS,True,CUST6950,2023,2023-10-01
4,INV1000004,2022-06-20,Abuja,Online,Student,Biscuits,FMCG,1,506.70,0.0,506.70,111.428571,454.73,454.73,60.80,POS,False,CUST3464,2022,2022-06-01


In [ ]:
# quick look at numbers of rows and columns
print('Datadf.shape)
print('---------' * 10)
# checking the structure of the dataset
df.info()
print('---------' * 10)
#basic descriptive stats for numeric columns
print(df.describe())

(50000, 20)
------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   InvoiceNo          50000 non-null  object 
 1   TransactionDate    50000 non-null  object 
 2   City               50000 non-null  object 
 3   Channel            50000 non-null  object 
 4   CustomerType       50000 non-null  object 
 5   Product            50000 non-null  object 
 6   Category           50000 non-null  object 
 7   Quantity           50000 non-null  int64  
 8   NominalUnitPrice   50000 non-null  float64
 9   DiscountRate       50000 non-null  float64
 10  NominalTotalValue  50000 non-null  float64
 11  CPI_Index          50000 non-null  float64
 12  RealUnitPrice      50000 non-null  float64
 13  RealTotalValue     50000 non-null  float64
 14  GrossMargin    